Load the Libraries we'll need

In [41]:
import os
import nltk
import pandas as pd
import math
import gzip
import numpy as np
import scipy.stats as stats
from scipy.sparse import csc_matrix, csr_matrix
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns; sns.set(style="white", color_codes=True)
import warnings 

%matplotlib inline
warnings.filterwarnings("ignore")


In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

#reviews
df = getDF('C:\\Users\\thomp\\Desktop\\HOD\\reviews_Patio_Lawn_and_Garden_5.json.gz')
df.to_csv('C:\\Users\\thomp\\Desktop\\HOD\\plg.csv', headers = True)
#products
df2 = getDF(r'C:\Users\thomp\Desktop\HOD\meta_Patio_Lawn_and_Garden.json.gz')
df2.to_csv(r'C:\Users\thomp\Desktop\HOD\metaplg.csv', headers = True)

In [41]:
df['asin'].describe()

count          13272
unique           962
top       B000A0VOD2
freq             296
Name: asin, dtype: object

In [6]:
#df2.head(5)
df2['asin'].describe()

count         109094
unique        109094
top       B0081JSDKQ
freq               1
Name: asin, dtype: object

In [58]:
df2.loc[df2['asin'] == 'B000A0VOD2']

,asin,description,title,imUrl,related,salesRank,categories,price,brand
6508,B000A0VOD2,Put an end to garden and landscaping damage on...,"Havahart Deer Off II DO32RTU Deer, Rabbit, and...",http://ecx.images-amazon.com/images/I/41jv1hiJ...,"{u'also_bought': [u'B000A09DT4', u'B0016PC4TS'...","{u'Patio, Lawn & Garden': 2686}","[[Patio, Lawn & Garden, Havahart, Repellents, ...",20.59,Havahart


In [59]:
df.loc[df['asin'] == 'B000A0VOD2']

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
1705,A3CW0ZLUO5X2B1,B000A0VOD2,"35-year Technology Consumer ""8-tracks to 802.11""","[0, 0]",1344729600,We've been using earlier versions of Deer Off ...,3.0,"08 12, 2012",Repellent properties are OK; battery powered s...
1706,A3SZXYX9YEDGT6,B000A0VOD2,"Addison Dewitt ""I'm nobody's fool.""","[1, 2]",1346284800,"Havahart DO64MS Deer Off II Deer, Rabbit, and ...",3.0,"08 30, 2012",Mixed Results
1707,A1SP366J07MNU,B000A0VOD2,"Adebisi A. Ilori ""beecee""","[0, 0]",1352937600,I live in an area where people leave their liv...,5.0,"11 15, 2012","great for stray goats, cows and donkeys"
1708,ADLVFFE4VBT8,B000A0VOD2,"A. Dent ""Aragorn""","[0, 0]",1343174400,Deer? (check)Rabbits? (check)Bears? (check)Tur...,4.0,"07 25, 2012","Deer hate it, so do bunnies"
1709,A319Y83RT0MRVR,B000A0VOD2,AIROLF,"[0, 0]",1366156800,This really works to keep the critters away fr...,5.0,"04 17, 2013",effective and easy to use
1710,AZYJE40XW6MFG,B000A0VOD2,"Alan Manning ""Gamairidium""","[1, 2]",1351123200,I got this repellent primarily to keep squirre...,4.0,"10 25, 2012",Effective
1711,A1W415JP5WEAJK,B000A0VOD2,Alex S,"[0, 0]",1358380800,We have experienced both squirrels and rabbits...,4.0,"01 17, 2013",Easy to use ...
1712,AV8MDYLHHTUOY,B000A0VOD2,"Amazon Customer ""eyegor""","[0, 0]",1366416000,"(Original Text: April 20, 2013)I hate how my s...",4.0,"04 20, 2013",Seems to stop ravenous rabbits in their tracks
1713,A3S0Y5GVW1CYRW,B000A0VOD2,"Amazon Customer ""geek girl""","[0, 0]",1342483200,We were anxious to try this product because we...,3.0,"07 17, 2012",Worked for some animals but not for others.
1714,A2ZT5LYQVJQJAU,B000A0VOD2,"Amazon Customer ""handspun knitter""","[0, 0]",1339113600,"First, be sure to read all the instructions on...",5.0,"06 8, 2012",It IS working !


In [42]:
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#convert into a list of words
#remove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thomp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
import csv
import re
from textblob import TextBlob

def senti(path, readfl, writefl):    
    readpath = path + '\\' + readfl
    writepath = path + '\\' + writefl    
    with open(readpath, 'r') as csvfile:
        reader = csv.reader(csvfile)
        with open(writepath, 'w') as output:
            writer = csv.writer(output)
            for col in reader:
                review = col[6]
                raw_sentences = tokenizer.tokenize(review)
                sentences = []
                for raw_sentence in raw_sentences:
                    if len(raw_sentence) > 0:
                        sentences.append(sentence_to_wordlist(raw_sentence))
                word_count = sum([len(sentence) for sentence in sentences])
                x = 0
                sentence_count = sum([x+1 for sentence in sentences])
                '''
                print sentences
                print("The review corpus contains {0:,} words".format(word_count))
                print("The review corpus contains {0:,} sentences".format(sentence_count))
                '''
                rowid = col[0]
                blob = TextBlob(review)
                if col[1] == 'reviewerID':
                #add column headers
                  writer.writerow(('rowID', col[1], col[2], col[3], col[4], col[5], col[6], col[7], col[8], col[9],
                                 'sentimentPolarity', 'sentimentSubjectivity', 'sentences', 'sentenceCount', 'wordCount'))
                else:
                  writer.writerow((col[0], col[1], col[2], col[3], col[4], col[5], col[6], col[7], col[8], col[9], 
                               blob.sentiment.polarity, blob.sentiment.subjectivity, sentences, sentence_count, word_count))
            output.close()            
    print("Done")
        
senti(r'C:\Users\thomp\Desktop\HOD', 'plg.csv', 'plgwithsentiment.csv')

Done


In [59]:
import multiprocessing

#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [61]:
import gensim.models.word2vec as w2v

reviews2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [67]:
sentences = []

with open(r'C:\Users\thomp\Desktop\HOD\plg.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for col in reader:
            review = col[6]
            raw_sentences = tokenizer.tokenize(review)
            for raw_sentence in raw_sentences:
                    if len(raw_sentence) > 0:
                        sentences.append(sentence_to_wordlist(raw_sentence))
                
word_count = sum([len(sentence) for sentence in sentences])
print(word_count)
print type(sentences)

2136529
<type 'list'>


In [68]:
reviews2vec.build_vocab(sentences)

In [38]:
df['reviewText'].describe()

count     13272
unique    13254
top            
freq         14
Name: reviewText, dtype: object

In [39]:
df['reviewText']

0        Good USA company that stands behind their prod...
1        This is a high quality 8 ply hose. I have had ...
2        It's probably one of the best hoses I've ever ...
3        I probably should have bought something a bit ...
4        I bought three of these 5/8-inch Flexogen hose...
5        I aqm impressed with the cost and quality of t...
6        After using this hose I will never go back to ...
7        This is one tough hose and it is thick and sti...
8        I'll buy more of these come  Spring. Holding u...
9        So far so good! I have not really had the time...
10       Hey Gilmore I have been buying your hoses for ...
11       This is a good quality hose that I use to conn...
12       I bought a cheap hose that didn't hold the pre...
13       I was worried about getting a short one, but t...
14       For everything I need for my lawn and garden w...
15       It's been outside in the hot (South Florida) s...
16       I've always been one of those people that boug.